In [22]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import os
import re
import copy
import json

In [23]:
from chessboard_utils import *

In [ ]:
game_state = np.array([
    [-2, -3, -4, -6, -5, -4, -3, -2],
    [-1, -1, -1, -1, -1, -1, -1, -1],
    [ 0,  0,  0,  0,  0,  0,  0,  -7],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  7],
    [ 1,  1,  1,  1,  1,  1,  1,  1],
    [ 2,  3,  4,  6,  5,  4,  3,  2],
])

    
draw_chessboard(game_state, show=True)


In [25]:
import sys
from pathlib import Path

project_root = Path().absolute().parent
sys.path.append(str(project_root))

from movement_analysis import *
from actualize_game_state import *

In [ ]:
with open('game_state.json', 'r') as f:
    game_data = json.load(f)
    
length = len(game_data['game_states'])    
length

In [ ]:
# Read the game states from game_state.json
# with open('challenge_game_state2.json', 'r') as f:
#     game_data = json.load(f)

with open('game_state_1.json', 'r') as f:
    game_data = json.load(f)


last_game_state = game_data['game_states'][0]['gs']

last_game_state = np.fliplr(last_game_state.copy())
last_game_state = np.rot90(last_game_state.copy(), k=-1)
last_game_state = np.rot90(last_game_state.copy(), k=-1)


draw_chessboard(last_game_state, show=True)

potential_castling = None

print('last_game_state : ', last_game_state)

actualized_game_state = initialize_game_state(last_game_state)

analyzed_data = {"game_state": []}
# Display each game state
for i, state in enumerate(game_data['game_states'][1:]):

    print('\n##############################################')
    print(f"\nGame state {i} (Frame: {state['frame']}):")
    # valid_move = True
    
    curr_state = np.array(state['gs'])

    curr_state = np.fliplr(curr_state.copy())
    curr_state = np.rot90(curr_state.copy(), k=-1)
    curr_state = np.rot90(curr_state.copy(), k=-1)


    move_analysis, potential_castling = analyze_move(last_game_state, curr_state, potential_castling, actualized_game_state)

    # valid_move = move_analysis['valid']
    
    if move_analysis['valid']:

        if move_analysis['move_type'] != 'castling' and move_analysis['error_pos'] is not None:

            error_pos = move_analysis['error_pos']
            print('Correct error position : ', error_pos)
            curr_state[error_pos] = last_game_state[error_pos]


        last_game_state = curr_state.copy()


        if move_analysis['move_type'] == 'castling':

            new_game_state, board, piece_certainty = actualize_game_state_with_castling(actualized_game_state, move_analysis, curr_state)
            print(board)

        else:

            actualized_game_state, board, piece_certainty = actualize_game_state(actualized_game_state, move_analysis, curr_state)
            print(board)

            board_state_serializable = board.tolist()

        # Save the current frame and board state
            analyzed_data["game_state"].append({
            "frame": state["frame"],
            "gs": board_state_serializable
        })
            from_pos = move_analysis['from_pos']
            to_pos = move_analysis['to_pos']
            piece = move_analysis['piece']

            valid_pices = move_analysis['valid_pieces']
            
            # Convertir les positions en notation d'échecs
            from_square = f"{chr(97 + from_pos[1])}{8 - from_pos[0]}"
            to_square = f"{chr(97 + to_pos[1])}{8 - to_pos[0]}"
            
            if move_analysis['move_type'] == 'move':
                print(f"Move: {from_square} -> {to_square}")
            
            else:  # capture
                print(f"Capture: {from_square} x {to_square}")


        print(piece_certainty)
        print(board)
        draw_chessboard(board, show=True)
        
    else:
        print(f"Invalid move: {move_analysis['message']}")

In [28]:
with open('analyzed_game_states.json', 'w') as f:
    json.dump(analyzed_data, f, indent=4)